In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import re
import os
from datetime import date
import mygene         #for alternative method of changing ensembl names to "normal" gene names
mg = mygene.MyGeneInfo()

In [45]:
#Function for performing the PCA. Takes dataframe with expression values as input
def my_pca(df, n_pc=1, normalize=True):
    df = df.dropna(axis = 0, how = 'all') #Remove rows with only NA values. Should be none
    x = df.values.T #Set x as transpose of only the numerical values of the dataframe
    if normalize:
        x2 = preprocessing.scale(x) #Standardize the data (center to mean and scale to unit variance)
    else:
        x2 = x
    pca = PCA(n_components = n_pc) #Set PCA parameters
    pca.fit(x2) #Fit data to model
    expl = pca.explained_variance_ratio_
    #my_pca.pca = pca #Used for Patrik's dictionary method for components
    x3 = pca.fit_transform(x2) #Transform the data (apply dimensionality reduciton) and set x3 as principal components 
    out_df = pd.DataFrame(x3.transpose(), index=list(range(1,n_pc+1)), columns=df.columns) #Create dataframe with vlues from the PCA and set columnindex as the PC number 
    out_df = out_df.transpose()
    return out_df, expl

#Function for creating csv with all TFs and their target genes. For each TF, identifies urls for target gene data and fetches the info and puts it into a list that is then converted to a dataframe and given as output
def TFs_targets(kbp, chip):   
    TFrange = ('Target Genes (TSS }'+kbp+'k)')
    TF_gene_list = []
    dex = 0
    for url in chip.loc[:,TFrange]: #OBS! This takes a long time.
        try:
            TF_gene_set = pd.read_csv(url, sep='\t') #For a specific TF, read csv from url as a dataframe
            genes = TF_gene_set['Target_genes'].tolist() #Take the contents of column 'Target_genes' and puts it into a list
            TF_gene_list.append(genes) #Append the list for a specific TF to list with all TFs
            print('Genes for '+chip.index[dex]+' ('+str(dex)+') found')
        except HTTPError: #If the url can't be reached, insert 'Not found' in the list and continue (to get correct index)
            genes = ['Not found']
            TF_gene_list.append(genes) #Append message that genes were not found
            print('Genes for '+chip.index[dex]+' ('+str(dex)+') NOT found')
        dex = dex+1
    TF_gene_sets = pd.DataFrame({'Genes':TF_gene_list}, index=chip.index) #Create a dataframe from the list of TFs and their target genes
    TF_gene_sets.to_csv('../data/Transfactors/TF_gene_sets.tsv', sep='\t')
    return TF_gene_sets

#Function for replacing the ensenbl gene IDs with gene symbols/names in expression data
def ID_to_symbol(datatsv):    
    dataset = pd.read_csv(datatsv, sep='\t') 
    symbols = pd.read_csv('../data/Gene_names.txt', index_col='Gene stable ID')
    symbols = symbols.drop_duplicates(keep=False)
    dataset = dataset.merge(symbols, left_on='Gene', right_on='Gene stable ID', how = 'left')
    dataset.dropna(how='any', inplace=True)
    dataset.set_index('Gene name', inplace=True)
    dataset.drop(columns='Gene', inplace=True)
    dataset.index.rename('Symbol', inplace=True)
    dataset.to_csv('../data/genes.raw.htseq2.symbolsTEST.tsv', sep='\t')
    return dataset

#Alternative function for replacing the ensembl gene IDs with gene symbols/names in expression data, using MyGene package.
def ID_to_symbolMG(datatsv):
    dataset = pd.read_csv(datatsv, sep='\t')
    genes = dataset.loc[:,'Gene']
    genesyms = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', as_dataframe=True) #Look up gene symbols with mygene
    genesyms = genesyms.dropna(subset=['symbol'], how='any') #Drop genes where no gene symbol was identified
    genesyms = genesyms.loc[~genesyms.index.duplicated(keep=False)] #Drop genes for which multiple gene symbols were identified
    syms = genesyms.loc[:,'symbol']
    dataset = dataset.merge(syms, left_on='Gene', right_on='query', how = 'left')
    dataset.dropna(how='any', inplace=True)
    dataset.set_index('symbol', inplace=True)
    dataset.index.rename('Symbol',inplace=True)
    dataset.drop(columns='Gene', inplace=True)
    dataset.to_csv('../data/genes.raw.htseq2.symbolsMG.tsv', sep='\t')
    return dataset

# The original chip_atlas_analysis_list.csv that was downloaded did not work, as it contained an extra '"' in the start and end of each row with '"' in it, and each '"' had an extra  '"'
def csv_fix(csv):
    name, ext = os.path.splitext(csv)
    newfilename = '{name}_{add}{ext}'.format(name=name, add='CORRECTED', ext='.csv')
    with open(csv,'r+') as csv_file:
        for line in csv_file:
            # removing starting and ending quotes of a line
            pattern1 = re.compile(r'^"|"$',re.MULTILINE)
            line = re.sub(r'^"|"$',"",line)
            # substituting escaped quote with a single quote
            pattern2 = re.compile(r'""')
            line = re.sub(r'""','"',line)
            corrected_csv = open(newfilename,'a')
            corrected_csv.write(line)
            corrected_csv.close()
    csv_df = pd.read_csv(newfilename, quotechar = '"')
    return csv_df

#Create dataframe with multi-colunindex of organ and developmental stage
def indexing(dataset, dataset2):
    #Exctract important characteristincs of samples from dataset2
    chars = pd.DataFrame()
    chars['assay'] = dataset2.loc[:,'Assay Name'].str.slice(stop=6)
    #chars['dev_stage'] = dataset2.loc[:,'Characteristics[developmental stage]']
    chars['organ'] = dataset2.loc[:,'Characteristics[organism part]']
    chars = chars.drop_duplicates()
    chars['dev_stage'] = [-5.5,-5.5,-2.5,-2.5,0.5,0.5,4,4,22,22,29,29,-5.5,-5.5,-2.5,-2.5,0.5,0.5,4,4,22,22,29,29]
    chars = chars.set_index('assay')

    #Insert the important characteristics into dataset with expression values as column index
    datasetT = dataset.T
    datasetT['dev_stage'] = datasetT.index.to_series().map(chars['dev_stage'])
    datasetT['organ'] = datasetT.index.to_series().map(chars['organ'])
    datasetT.set_index(['organ','dev_stage'], inplace=True)
    #expdata = datasetT.sort_index(level = 0).T
    expdata = datasetT.T
    return expdata

#Remove genes (rows) with 0 activity at any timepoint and log2 transform the data, or add constant (1) and log2 transform
def log2trans(expdata):
    #expdata = expdata.loc[~(expdata<=0.0).any(axis=1)]
    expdata = expdata+1
    logdata = np.log2(expdata)
    display(logdata)
    return logdata

#Performing the PCA
def run_my_pca(expdata, TF_gene_sets):
    var_expl = []
    PCA_per_TF = pd.DataFrame(index=expdata.columns)
    for TF in TF_gene_sets.index: 
        genesIndex = pd.DataFrame(TF_gene_sets.loc[TF,'Genes'], columns=['Genes'])
        genesIndex.set_index('Genes', inplace=True)  
        genes = genesIndex.index.intersection(expdata.index)
        TFdata = expdata.loc[genes,:]
        [res, expl] = my_pca(TFdata)
        PCA_per_TF[TF] = res
        var_expl.append([TF,expl[0],len(genes)])

    df_var_expl = pd.DataFrame(var_expl, columns=['TF','Variance explained', 'Nr of genes'])
    df_var_expl = df_var_expl.set_index('TF')
    PCA_per_TF.sort_index(inplace=True)
    display(PCA_per_TF)
    display(df_var_expl)
    return PCA_per_TF, df_var_expl

#Save data from performed experiments as tsv file
def save_exp(data, filename):
    if os.path.isdir('../exp/'+str(date.today())):
        data.to_csv('../exp/'+str(date.today())+'/'+filename+'.tsv', sep='\t')
    else:
        os.mkdir('../exp/'+str(date.today()))
        data.to_csv('../exp/'+str(date.today())+'/'+filename+'.tsv', sep='\t')

In [51]:
# Read csv with all all TFs and urls to files with their target genes
try:
    chip = pd.read_csv('../data/Transfactors/chip_atlas_analysis_list_CORRECTED.csv').loc[1255:1953,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
except FileNotFoundError:
    chip = csv_fix('../data/Transfactors/chip_atlas_analysis_list.csv').loc[1255:1953,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
chip = chip.set_index('Antigen')

#Read csv with TFs and their target genes. If not available, fetch the data and create a csv
try: 
    TF_gene_sets = pd.read_csv('../data/Transfactors/TF_gene_sets.tsv', sep='\t', index_col='Antigen', converters={'Genes': lambda x: x.strip('[]').replace("'","").split(', ')})
except FileNotFoundError:
    TF_gene_sets = TFs_targets('10', chip)

#Read in dataset with expression values and gene symbols. If not available, replace ensembl IDs and create the csv
try:
    dataset = pd.read_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t', index_col='Symbol')
except:
    ID_to_symbol('../data/genes.raw.htseq2.tsv')

#Read in dataset with sample sepcifications
dataset2 = pd.read_csv('../data/E-MTAB-2328.sdrf.tsv', sep='\t') #Read csv with specifications of assays

In [52]:
expdata = indexing(dataset,dataset2)
logdata = log2trans(expdata)

organ            liver                 brain                 liver             \
dev_stage        -5.5       -5.5       -5.5       -5.5       -2.5       -2.5    
Symbol                                                                          
Gnai3        12.229119  10.685625  11.294046  10.476746  12.222795  11.764457   
Pbsn          0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Cdc45         9.943980   8.400879   7.357552   6.768184   9.103288   8.939579   
H19          16.244364  14.945535  12.006326  10.863412  17.475726  17.104548   
Scml2         6.189825   4.700440   6.426265   5.781360   5.614710   5.285402   
...                ...        ...        ...        ...        ...        ...   
Vmn1r-ps47    0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Gm22396       0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Gm22394       0.000000   0.000000   0.000000   0.000000   1.000000   0.000000   
Vmn1r-ps147   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Methig1       0.000000   0.000000   0.000000   0.000000   0.000000   2.321928   

organ            brain                 liver             ...      brain  \
dev_stage        -2.5       -2.5        0.5        0.5   ...       4.0    
Symbol                                                   ...              
Gnai3        10.727070  11.295195  11.277287  10.153552  ...  11.246741   
Pbsn          0.000000   0.000000   0.000000   0.000000  ...   0.000000   
Cdc45         6.000000   6.507795   8.717676   7.665336  ...   6.643856   
H19          10.333155  11.711237  16.273267  16.137952  ...   8.658211   
Scml2         6.000000   6.882643   5.129283   4.169925  ...   6.539159   
...                ...        ...        ...        ...  ...        ...   
Vmn1r-ps47    0.000000   0.000000   0.000000   0.000000  ...   0.000000   
Gm22396       0.000000   0.000000   0.000000   0.000000  ...   0.000000   
Gm22394       0.000000   0.000000   0.000000   0.000000  ...   0.000000   
Vmn1r-ps147   0.000000   0.000000   0.000000   0.000000  ...   0.000000   
Methig1       0.000000   0.000000   1.000000   0.000000  ...   0.000000   

organ                       liver                 brain                 liver  \
dev_stage         4.0        22.0       22.0       22.0       22.0       29.0   
Symbol                                                                          
Gnai3        10.553629  11.053926  10.947637  10.172428  10.461479  10.592457   
Pbsn          0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Cdc45         6.954196   6.629357   6.794416   5.392317   5.357552   4.807355   
H19           9.961450  11.116344  11.220378   5.129283   6.209453   7.247928   
Scml2         6.000000   4.169925   2.584963   3.807355   4.169925   2.807355   
...                ...        ...        ...        ...        ...        ...   
Vmn1r-ps47    0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Gm22396       0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Gm22394       0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Vmn1r-ps147   0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   
Methig1       0.000000   0.000000   0.000000   0.000000   0.000000   0.000000   

organ                       brain            
dev_stage         29.0       29.0      29.0  
Symbol                                       
Gnai3        10.522582  10.920353  9.743151  
Pbsn          0.000000   0.000000  0.000000  
Cdc45         5.044394   6.209453  4.906891  
H19           6.169925   5.584963  4.906891  
Scml2         1.584963   4.754888  3.321928  
...                ...        ...       ...  
Vmn1r-ps47    0.000000   0.000000  0.000000  
Gm22396       0.000000   0.000000  0.000000  
Gm22394       0.000000   0.000000  0.000000  
Vmn1r-ps147   0.000000   0.000000  0.000000  
Methig1       0.000000   0.000000  0.000000  

[32394 rows x 24 columns]

In [11]:
[PCA_per_TF, df_var_expl] = run_my_pca(logdata, TF_gene_sets)

Acaa2      Acss2      Actb       Adnp       Aebp2  \
organ dev_stage                                                          
brain -5.5        6.226281   8.529159  1.266096  27.803163   48.911427   
      -5.5      -16.026141 -11.080162 -2.560023  -9.080403   -8.399281   
      -2.5       -6.415215  -1.315675  0.238415  11.970542   23.030566   
      -2.5       15.509589  19.226379  3.846335  48.199373   80.695728   
       0.5       -1.900806   4.408854  1.569242  22.089774   39.101975   
       0.5        6.089299  11.628409  2.293904  34.319519   58.494813   
       4.0       12.870728  18.250992  3.847518  47.182314   77.568465   
       4.0       -0.864701   5.399897  1.637835  22.169094   40.491036   
       22.0       9.354045  16.916814  2.585774  38.356126   65.929095   
       22.0      20.188988  26.123048  4.924583  57.086975   93.902975   
       29.0      36.314267  39.720863  7.508891  83.351472  133.489390   
       29.0       5.155627  12.490148  2.442148  32.159110   56.307690   
liver -5.5       14.264697   3.115055  0.226030  -6.669647  -16.835888   
      -5.5      -27.342495 -33.933737 -6.367893 -69.896608 -114.094041   
      -2.5       10.575421   3.195663 -0.939467 -10.509340  -19.181692   
      -2.5        3.825196  -3.301216 -1.427344 -19.124875  -34.123670   
       0.5        0.310541  -5.097323 -1.378283 -23.580253  -40.062108   
       0.5      -26.142681 -29.525045 -5.083790 -63.054223 -102.780737   
       4.0       27.039052  19.640883  2.527302  20.594890   28.443422   
       4.0       -4.712470  -8.451385 -2.512193 -30.159002  -48.617466   
       22.0     -12.388537 -16.662337 -2.036337 -40.295380  -69.856087   
       22.0     -17.303901 -20.938644 -3.353647 -44.669920  -77.986267   
       29.0     -23.379348 -25.697579 -4.185810 -58.680280  -99.222484   
       29.0     -31.247437 -32.643059 -5.069284 -69.562421 -115.206860   

                       Aff3       Aff4       Ahcy        Ahr      Aicda  ...  \
organ dev_stage                                                          ...   
brain -5.5        39.750826  13.551555   4.445386  21.880013   5.529695  ...   
      -5.5       -17.182138 -23.721013  -2.161922   7.404864  -2.779463  ...   
      -2.5        11.740111  -7.714021   1.498693  15.957010   2.470233  ...   
      -2.5        69.869236  30.179012   9.489210  31.464502  11.372489  ...   
       0.5        26.394271   0.573338   3.097462  20.599661   5.829620  ...   
       0.5        46.977780  14.551811   6.760040  26.932549   8.427793  ...   
       4.0        63.945339  25.618714   8.326973  31.014257  10.784911  ...   
       4.0        27.176457   2.534182   5.018471  19.038340   4.304829  ...   
       22.0       52.265260  19.205241   7.795133  25.819286  10.680901  ...   
       22.0       81.269989  37.431279  11.075787  33.217164  14.831416  ...   
       29.0      122.523017  64.360871  16.789923  43.257033  21.141966  ...   
       29.0       42.437433  12.678929   6.700513  23.492050   9.093698  ...   
liver -5.5        -2.751033  17.610701  -1.910194 -10.984680  -4.518835  ...   
      -5.5      -102.792234 -51.163551 -13.872510 -36.963512 -19.944806  ...   
      -2.5        -7.732026  13.475352  -2.114143 -14.121705  -4.746649  ...   
      -2.5       -22.996027   2.313260  -4.091766 -18.307753  -7.308553  ...   
       0.5       -29.260273  -3.661320  -3.998451 -20.020864  -7.576968  ...   
       0.5       -92.682704 -47.985573 -11.425727 -36.392489 -16.565943  ...   
       4.0        39.967248  42.914141   3.667743  -2.651387   4.536263  ...   
       4.0       -39.400352 -10.779168  -5.592965 -23.334521  -8.561603  ...   
       22.0      -56.125235 -23.667092  -7.204597 -29.121992  -6.510477  ...   
       22.0      -65.910646 -31.015162  -8.753545 -31.195417  -8.124572  ...   
       29.0      -85.136373 -42.130846 -10.976809 -35.437520  -9.924695  ...   
       29.0     -102.347926 -55.160640 -12.562702 -41.544889 -12.441250  ...   

               

,Variance explained,Nr of genes
TF,,
Acaa2,0.487604,613
Acss2,0.493546,781
Actb,0.422738,32
Adnp,0.422879,4361
Aebp2,0.471388,10761
...,...,...
Zic3,0.452157,4240
Zkscan1,0.425477,5528
Zmiz1,0.433857,11167


In [58]:
logdata.loc['Rtf1']

organ  dev_stage
liver  -5.5          9.377211
       -5.5          8.076816
brain  -5.5          9.781360
       -5.5          9.515700
liver  -2.5          8.960002
       -2.5          8.897845
brain  -2.5          8.625709
       -2.5         10.474720
liver   0.5          9.011227
        0.5          8.055282
brain   0.5          9.172428
        0.5         10.424166
liver   4.0          9.896332
        4.0          8.839204
brain   4.0          9.493855
        4.0          9.649256
liver   22.0         7.870365
        22.0         7.357552
brain   22.0         9.712527
        22.0         9.840778
liver   29.0         7.707359
        29.0         7.607330
brain   29.0        10.245553
        29.0         9.667112
Name: Rtf1, dtype: float64

In [57]:
logdata.loc['Hnf1a',:]

organ  dev_stage
liver  -5.5         6.807355
       -5.5         6.000000
brain  -5.5         0.000000
       -5.5         1.000000
liver  -2.5         7.442943
       -2.5         7.523562
brain  -2.5         3.321928
       -2.5         1.584963
liver   0.5         7.643856
        0.5         7.400879
brain   0.5         2.321928
        0.5         2.000000
liver   4.0         8.426265
        4.0         7.321928
brain   4.0         3.584963
        4.0         1.000000
liver   22.0        8.566054
        22.0        8.228819
brain   22.0        3.459432
        22.0        3.321928
liver   29.0        8.005625
        29.0        7.977280
brain   29.0        4.857981
        29.0        3.000000
Name: Hnf1a, dtype: float64

In [12]:
df_var_expl.nlargest(10, ['Variance explained'])

,Variance explained,Nr of genes
TF,,
Thoc5,0.970547,3
Sp110,0.676744,10
Snai1,0.628385,604
Art1,0.612289,8
Gcm1,0.606397,8
Eed,0.577656,2988
Nlrp3,0.545675,5
Setd2,0.543182,7
Mtf2,0.529785,6027


In [46]:
save_exp(PCA_per_TF, 'PCA_results')
save_exp(df_var_expl, 'Variance_explained')